# Wuhan Outbound Flights Analysis

**Objective**: to visualize the outbound flight and enable in-depth analysis

**Data source**: history.adsbexchange.com 

https://history.adsbexchange.com/downloads/samples/?fbclid=IwAR0vGBAUsyZcPa-7pqRN13zK6Hg_eeQ6hAlSYCm84ZpmDBsr5ld4zj3MxNc



**DataDictionary**: https://history.adsbexchange.com/datafields/


**Data**: **1st Jan 2020 only** outbound of Wuhan Tianhe Airport (WUH) ( other date will be added in later)

**Output sample dashboard**: to-be-update


In [1]:
import time
import re
import numpy as np
import pandas as pd
import pickle as pkl
import math
import json
import warnings

import datetime
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./Data/wuhan_outbound_01012020.csv')

airport =  pd.read_csv('./airport.csv')



In [3]:
df.head()

,Alt,AltT,Bad,CMsgs,CNum,Call,CallSus,Cos,Cou,EngMount,EngType,Engines,FSeen,FlightsCount,From,GAlt,HasPic,HasSig,Help,Icao,Id,Interested,Lat,Long,Man,Mdl,Mil,Mlat,Op,OpIcao,PosTime,Rcvr,Reg,ResetTrail,Spd,SpdTyp,Species,Sqk,TSecs,TT,Tisb,To,Trak,TrkH,Trt,Type,Vsi,VsiT,WTC,Year,Stops,Gnd,InHg,Sig,PosStale,TAlt,TTrk,Tag
0,3000.0,0.0,0.0,19.0,63065.0,CSH9362,0.0,"[31.2624, 121.2577, 1577884849652.0, 4000.0, 3...",China,0.0,3.0,2.0,/Date(1577884674266)/,6.0,"ZHHH Wuhan Tianhe, China",3000.0,0.0,0.0,0.0,781374,7869300.0,0.0,31.31760,121.24890,Boeing,Boeing 737NG 800/W,0.0,0.0,Shanghai Airlines,CSH,1.577885e+12,1011157.0,B-1265,1.0,195.0,0.0,1.0,6607.0,273.0,a,0.0,"ZSPD Shanghai Pudong, China",356.0,0.0,2.0,B738,-1408.0,0.0,2.0,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25600.0,0.0,0.0,16.0,43225.0,CSN660,0.0,"[27.63885, 113.42638, 1577884867961.0, 25500.0...",China,0.0,3.0,2.0,/Date(1577884000540)/,10.0,"ZHHH Wuhan Tianhe, China",25600.0,0.0,0.0,0.0,780E06,7867910.0,0.0,27.54648,113.44995,Boeing,Boeing 777 31BER,0.0,0.0,China Southern Airlines,CSN,1.577885e+12,1011157.0,B-2049,1.0,853.0,0.0,1.0,316.0,947.0,a,0.0,"ZGGG Guangzhou Baiyun, China",164.0,0.0,2.0,B773,0.0,0.0,3.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25600.0,0.0,0.0,2.0,43225.0,CSN660,0.0,"[27.63885, 113.42638, 1577884863206.0, None, 2...",China,0.0,3.0,2.0,/Date(1577884863206)/,0.0,"ZHHH Wuhan Tianhe, China",25600.0,0.0,0.0,0.0,780E06,7867910.0,0.0,27.54648,113.44995,Boeing,Boeing 777 31BER,0.0,0.0,China Southern Airlines,CSN,1.577885e+12,1021303.0,B-2049,1.0,853.0,0.0,1.0,316.0,85.0,a,0.0,"ZGGG Guangzhou Baiyun, China",164.0,0.0,2.0,B773,0.0,0.0,3.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,800.0,0.0,0.0,5.0,41473.0,CES2483,0.0,"[22.74843, 113.73332, 1577884832536.0, 2000.0,...",China,0.0,3.0,2.0,/Date(1577884772675)/,0.0,"ZHHH Wuhan Tianhe, China",800.0,0.0,0.0,0.0,780CAC,7867564.0,0.0,22.70879,113.75509,Boeing,Boeing 737NG 89P/W,0.0,0.0,China Eastern Airlines,CES,1.577885e+12,1021303.0,B-1965,1.0,274.0,0.0,1.0,322.0,176.0,a,0.0,"ZJHK Haikou Meilan, China",152.0,0.0,2.0,B738,-832.0,0.0,2.0,2014.0,"[""ZGSZ Shenzhen Bao'an, China""]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3000.0,0.0,0.0,43.0,63065.0,CSH9362,0.0,"[31.2624, 121.2577, 1577884849864.0, 4000.0, 3...",China,0.0,3.0,2.0,/Date(1577883939492)/,0.0,"ZHHH Wuhan Tianhe, China",3000.0,0.0,0.0,0.0,781374,7869300.0,0.0,31.31760,121.24890,Boeing,Boeing 737NG 800/W,0.0,0.0,Shanghai Airlines,CSH,1.577885e+12,1021303.0,B-1265,1.0,195.0,0.0,1.0,6607.0,1009.0,a,0.0,"ZSPD Shanghai Pudong, China",356.0,0.0,2.0,B738,-1408.0,0.0,2.0,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.columns

Index(['Alt', 'AltT', 'Bad', 'CMsgs', 'CNum', 'Call', 'CallSus', 'Cos', 'Cou',
       'EngMount', 'EngType', 'Engines', 'FSeen', 'FlightsCount', 'From',
       'GAlt', 'HasPic', 'HasSig', 'Help', 'Icao', 'Id', 'Interested', 'Lat',
       'Long', 'Man', 'Mdl', 'Mil', 'Mlat', 'Op', 'OpIcao', 'PosTime', 'Rcvr',
       'Reg', 'ResetTrail', 'Spd', 'SpdTyp', 'Species', 'Sqk', 'TSecs', 'TT',
       'Tisb', 'To', 'Trak', 'TrkH', 'Trt', 'Type', 'Vsi', 'VsiT', 'WTC',
       'Year', 'Stops', 'Gnd', 'InHg', 'Sig', 'PosStale', 'TAlt', 'TTrk',
       'Tag'],
      dtype='object')

In [4]:
airport.head()

,AirportID,Name,City,Counry,IATA,ICAO,Latitude,Longtitude,Altitude,timezone,DSY,tz_database_timezone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [5]:
## create mapping dict 
icao_iata_dict = airport.set_index('ICAO')['IATA'].to_dict()


In [9]:
def getToCountry(row):
    out = row.split(', ')
    return out[-1]


def getCity(row):
    out = row.split(' ')
    return out[-1]


def getICAOCode(row):
    out = row.split(' ')
    return out[0]

def getCityCode(row):
    out = row.split(' ')
    return out[0]

    

In [10]:
df['To_country'] = df['To'].apply(getToCountry)

In [11]:
df['To_city'] = df['To'].apply(getCity)
df['From_city'] = df['From'].apply(getCity)



In [12]:
df['ICAO_code_from'] = df['From'].apply(getICAOCode)
df['ICAO_code_to'] = df['To'].apply(getICAOCode)


In [13]:
df['ICAO_code_from'].head()

0    ZHHH
1    ZHHH
2    ZHHH
3    ZHHH
4    ZHHH
Name: ICAO_code_from, dtype: object

In [14]:
df['IATA_code_from'] = df['ICAO_code_from'].map(icao_iata_dict)
df['IATA_code_to'] = df['ICAO_code_to'].map(icao_iata_dict)


In [15]:
df.head()

,Alt,AltT,Bad,CMsgs,CNum,Call,CallSus,Cos,Cou,EngMount,EngType,Engines,FSeen,FlightsCount,From,GAlt,HasPic,HasSig,Help,Icao,Id,Interested,Lat,Long,Man,Mdl,Mil,Mlat,Op,OpIcao,PosTime,Rcvr,Reg,ResetTrail,Spd,SpdTyp,Species,Sqk,TSecs,TT,Tisb,To,Trak,TrkH,Trt,Type,Vsi,VsiT,WTC,Year,Stops,Gnd,InHg,Sig,PosStale,TAlt,TTrk,Tag,To_country,To_city,From_city,ICAO_code_from,ICAO_code_to,IATA_code_from,IATA_code_to
0,3000.0,0.0,0.0,19.0,63065.0,CSH9362,0.0,"[31.2624, 121.2577, 1577884849652.0, 4000.0, 3...",China,0.0,3.0,2.0,/Date(1577884674266)/,6.0,"ZHHH Wuhan Tianhe, China",3000.0,0.0,0.0,0.0,781374,7869300.0,0.0,31.31760,121.24890,Boeing,Boeing 737NG 800/W,0.0,0.0,Shanghai Airlines,CSH,1.577885e+12,1011157.0,B-1265,1.0,195.0,0.0,1.0,6607.0,273.0,a,0.0,"ZSPD Shanghai Pudong, China",356.0,0.0,2.0,B738,-1408.0,0.0,2.0,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,China,China,ZHHH,ZSPD,WUH,PVG
1,25600.0,0.0,0.0,16.0,43225.0,CSN660,0.0,"[27.63885, 113.42638, 1577884867961.0, 25500.0...",China,0.0,3.0,2.0,/Date(1577884000540)/,10.0,"ZHHH Wuhan Tianhe, China",25600.0,0.0,0.0,0.0,780E06,7867910.0,0.0,27.54648,113.44995,Boeing,Boeing 777 31BER,0.0,0.0,China Southern Airlines,CSN,1.577885e+12,1011157.0,B-2049,1.0,853.0,0.0,1.0,316.0,947.0,a,0.0,"ZGGG Guangzhou Baiyun, China",164.0,0.0,2.0,B773,0.0,0.0,3.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,China,China,ZHHH,ZGGG,WUH,CAN
2,25600.0,0.0,0.0,2.0,43225.0,CSN660,0.0,"[27.63885, 113.42638, 1577884863206.0, None, 2...",China,0.0,3.0,2.0,/Date(1577884863206)/,0.0,"ZHHH Wuhan Tianhe, China",25600.0,0.0,0.0,0.0,780E06,7867910.0,0.0,27.54648,113.44995,Boeing,Boeing 777 31BER,0.0,0.0,China Southern Airlines,CSN,1.577885e+12,1021303.0,B-2049,1.0,853.0,0.0,1.0,316.0,85.0,a,0.0,"ZGGG Guangzhou Baiyun, China",164.0,0.0,2.0,B773,0.0,0.0,3.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,China,China,ZHHH,ZGGG,WUH,CAN
3,800.0,0.0,0.0,5.0,41473.0,CES2483,0.0,"[22.74843, 113.73332, 1577884832536.0, 2000.0,...",China,0.0,3.0,2.0,/Date(1577884772675)/,0.0,"ZHHH Wuhan Tianhe, China",800.0,0.0,0.0,0.0,780CAC,7867564.0,0.0,22.70879,113.75509,Boeing,Boeing 737NG 89P/W,0.0,0.0,China Eastern Airlines,CES,1.577885e+12,1021303.0,B-1965,1.0,274.0,0.0,1.0,322.0,176.0,a,0.0,"ZJHK Haikou Meilan, China",152.0,0.0,2.0,B738,-832.0,0.0,2.0,2014.0,"[""ZGSZ Shenzhen Bao'an, China""]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,China,China,ZHHH,ZJHK,WUH,HAK
4,3000.0,0.0,0.0,43.0,63065.0,CSH9362,0.0,"[31.2624, 121.2577, 1577884849864.0, 4000.0, 3...",China,0.0,3.0,2.0,/Date(1577883939492)/,0.0,"ZHHH Wuhan Tianhe, China",3000.0,0.0,0.0,0.0,781374,7869300.0,0.0,31.31760,121.24890,Boeing,Boeing 737NG 800/W,0.0,0.0,Shanghai Airlines,CSH,1.577885e+12,1021303.0,B-1265,1.0,195.0,0.0,1.0,6607.0,1009.0,a,0.0,"ZSPD Shanghai Pudong, China",356.0,0.0,2.0,B738,-1408.0,0.0,2.0,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,China,China,ZHHH,ZSPD,WUH,PVG


In [45]:
df.To_city

0            ZSPD Shanghai Pudong, China
1           ZGGG Guangzhou Baiyun, China
2           ZGGG Guangzhou Baiyun, China
3              ZJHK Haikou Meilan, China
4            ZSPD Shanghai Pudong, China
                      ...               
6681         ZBAA Beijing Capital, China
6682         ZSPD Shanghai Pudong, China
6683         ZBAA Beijing Capital, China
6684         ZBAA Beijing Capital, China
6685    WSSS Singapore Changi, Singapore
Name: To_city, Length: 6686, dtype: object

In [44]:
df.head(3)

,Alt,AltT,Bad,CMsgs,CNum,Call,CallSus,Cos,Cou,EngMount,EngType,Engines,FSeen,FlightsCount,From,GAlt,HasPic,HasSig,Help,Icao,Id,Interested,Lat,Long,Man,Mdl,Mil,Mlat,Op,OpIcao,PosTime,Rcvr,Reg,ResetTrail,Spd,SpdTyp,Species,Sqk,TSecs,TT,Tisb,To,Trak,TrkH,Trt,Type,Vsi,VsiT,WTC,Year,Stops,Gnd,InHg,Sig,PosStale,TAlt,TTrk,Tag,To_country,PosDateTime,To_city,From_city
0,3000.0,0.0,0.0,19.0,63065.0,CSH9362,0.0,"[31.2624, 121.2577, 1577884849652.0, 4000.0, 3...",China,0.0,3.0,2.0,/Date(1577884674266)/,6.0,"ZHHH Wuhan Tianhe, China",3000.0,0.0,0.0,0.0,781374,7869300.0,0.0,31.31760,121.24890,Boeing,Boeing 737NG 800/W,0.0,0.0,Shanghai Airlines,CSH,1.577885e+12,1011157.0,B-1265,1.0,195.0,0.0,1.0,6607.0,273.0,a,0.0,"ZSPD Shanghai Pudong, China",356.0,0.0,2.0,B738,-1408.0,0.0,2.0,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,2020-01-01 13:21:51.310,"ZSPD Shanghai Pudong, China","ZHHH Wuhan Tianhe, China"
1,25600.0,0.0,0.0,16.0,43225.0,CSN660,0.0,"[27.63885, 113.42638, 1577884867961.0, 25500.0...",China,0.0,3.0,2.0,/Date(1577884000540)/,10.0,"ZHHH Wuhan Tianhe, China",25600.0,0.0,0.0,0.0,780E06,7867910.0,0.0,27.54648,113.44995,Boeing,Boeing 777 31BER,0.0,0.0,China Southern Airlines,CSN,1.577885e+12,1011157.0,B-2049,1.0,853.0,0.0,1.0,316.0,947.0,a,0.0,"ZGGG Guangzhou Baiyun, China",164.0,0.0,2.0,B773,0.0,0.0,3.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,2020-01-01 13:22:09.775,"ZGGG Guangzhou Baiyun, China","ZHHH Wuhan Tianhe, China"
2,25600.0,0.0,0.0,2.0,43225.0,CSN660,0.0,"[27.63885, 113.42638, 1577884863206.0, None, 2...",China,0.0,3.0,2.0,/Date(1577884863206)/,0.0,"ZHHH Wuhan Tianhe, China",25600.0,0.0,0.0,0.0,780E06,7867910.0,0.0,27.54648,113.44995,Boeing,Boeing 777 31BER,0.0,0.0,China Southern Airlines,CSN,1.577885e+12,1021303.0,B-2049,1.0,853.0,0.0,1.0,316.0,85.0,a,0.0,"ZGGG Guangzhou Baiyun, China",164.0,0.0,2.0,B773,0.0,0.0,3.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,2020-01-01 13:22:06.952,"ZGGG Guangzhou Baiyun, China","ZHHH Wuhan Tianhe, China"


explanation of the columns can be found at: https://www.adsbexchange.com/datafields/

In [ ]:
df.head()

In [13]:
## descrptive stats
df.describe()

,Alt,AltT,Bad,CMsgs,CNum,CallSus,EngMount,EngType,Engines,FlightsCount,GAlt,HasPic,HasSig,Help,Id,Interested,Lat,Long,Mil,Mlat,PosTime,Rcvr,ResetTrail,Spd,SpdTyp,Species,Sqk,TSecs,Tisb,Trak,TrkH,Trt,Vsi,VsiT,WTC,Year,Gnd,InHg,Sig,PosStale,TAlt,TTrk
count,6683.000000,6686.0,6686.0,6686.000000,6.558000e+03,6686.000000,6686.0,6686.000000,6558.0,6686.000000,6683.000000,6686.0,6686.000000,5806.0,6.686000e+03,6686.0,6675.000000,6675.000000,6686.0,6675.0,6.675000e+03,6.686000e+03,6675.0,6684.000000,6686.000000,6686.000000,5806.000000,6686.000000,6686.0,6681.000000,6686.000000,6686.000000,6683.000000,6686.000000,6686.000000,6558.000000,1775.000000,2593.000000,1137.000000,28.0,687.000000,129.000000
mean,24610.552596,0.0,0.0,2241.333533,3.371974e+05,0.004038,0.0,2.942567,2.0,2.589141,24689.280413,0.0,0.170057,0.0,7.928339e+06,0.0,33.188910,100.186874,0.0,0.0,1.577875e+12,6.972839e+05,1.0,560.276825,0.004188,0.980856,3982.676025,1165.075531,0.0,163.702934,0.002094,2.304816,-21.847673,0.028268,2.244840,2011.863220,0.002254,30.122792,77.418646,1.0,30907.831150,169.671148
std,11109.005888,0.0,0.0,7401.023909,2.415288e+06,0.063424,0.0,0.411129,0.0,4.328514,11113.070409,0.0,0.375711,0.0,2.669120e+05,0.0,11.319929,40.338370,0.0,0.0,2.090244e+07,4.293796e+05,0.0,228.093754,0.091430,0.137043,1815.503904,1537.831816,0.0,100.487296,0.045715,0.899662,975.610552,0.165750,0.547545,4.400118,0.047431,0.216118,60.719764,0.0,10607.287504,117.106031
min,-475.000000,0.0,0.0,1.000000,3.380000e+02,0.000000,0.0,0.000000,2.0,0.000000,-475.000000,0.0,0.000000,0.0,7.454084e+06,0.0,-5.937256,-175.473907,0.0,0.0,1.577837e+12,1.115900e+04,1.0,1.500000,0.000000,0.000000,57.000000,2.000000,0.0,0.000000,0.000000,1.000000,-4096.000000,0.000000,0.000000,1999.000000,0.000000,28.789370,4.000000,1.0,1600.000000,0.000000
25%,18000.000000,0.0,0.0,12.000000,6.169000e+03,0.000000,0.0,3.000000,2.0,0.000000,18100.000000,0.0,0.000000,0.0,7.865599e+06,0.0,25.909672,109.401600,0.0,0.0,1.577860e+12,1.633180e+05,1.0,420.000000,0.000000,1.000000,3155.000000,263.000000,0.0,69.600000,0.000000,2.000000,-64.000000,0.000000,2.000000,2011.000000,0.000000,29.908085,29.000000,1.0,27616.000000,71.015625
50%,26525.000000,0.0,0.0,54.000000,3.492300e+04,0.000000,0.0,3.000000,2.0,0.000000,26550.000000,0.0,0.000000,0.0,7.866533e+06,0.0,31.358690,113.845410,0.0,0.0,1.577873e+12,1.011157e+06,1.0,492.000000,0.000000,1.000000,4232.000000,648.000000,0.0,179.000000,0.000000,2.000000,0.000000,0.000000,2.000000,2012.000000,0.000000,30.118110,55.000000,1.0,38016.000000,189.843750
75%,34000.000000,0.0,0.0,407.750000,3.894500e+04,0.000000,0.0,3.000000,2.0,6.000000,34000.000000,0.0,0.000000,0.0,7.868342e+06,0.0,37.474621,116.561156,0.0,0.0,1.577895e+12,1.021303e+06,1.0,764.000000,0.000000,1.000000,5375.000000,1400.000000,0.0,242.000000,0.000000,2.000000,0.000000,0.000000,3.000000,2015.000000,0.000000,30.295275,116.000000,1.0,38016.000000,270.000000
max,65286.000000,0.0,0.0,60744.000000,1.900073e+07,1.000000,0.0,3.000000,2.0,18.000000,65750.000000,0.0,1.000000,0.0,9.015365e+06,0.0,62.380883,140.916245,0.0,0.0,1.577923e+12,1.033151e+06,1.0,2657.000000,2.000000,1.000000,7744.000000,10305.000000,0.0,359.000000,1.000000,5.000000,4389.000000,1.000000,3.000000,2019.000000,1.000000,30.531496,255.000000,1.0,40000.000000,359.296875


In [14]:
## check null value
df.isnull().sum()

Alt                3
AltT               0
Bad                0
CMsgs              0
CNum             128
Call               0
CallSus            0
Cos               11
Cou                0
EngMount           0
EngType            0
Engines          128
FSeen              0
FlightsCount       0
From               0
GAlt               3
HasPic             0
HasSig             0
Help             880
Icao               0
Id                 0
Interested         0
Lat               11
Long              11
Man              128
Mdl              128
Mil                0
Mlat              11
Op               128
OpIcao           128
PosTime           11
Rcvr               0
Reg              128
ResetTrail        11
Spd                2
SpdTyp             0
Species            0
Sqk              880
TSecs              0
TT                 0
Tisb               0
To                 0
Trak               5
TrkH               0
Trt                0
Type             128
Vsi                3
VsiT         

In [16]:
## convert PosTime epoch mili-second timestamp to datetime
df['PosDateTime'] = pd.to_datetime(df['PosTime'],unit='ms')

In [17]:
df['PosDateTime'].tail()

6681   2020-01-01 02:33:04.046
6682   2020-01-01 02:32:33.308
6683   2020-01-01 02:33:04.942
6684   2020-01-01 02:33:02.815
6685   2020-01-01 21:00:31.995
Name: PosDateTime, dtype: datetime64[ns]

In [96]:
## unique count of flight id
len(set(df.Id))

74

In [18]:
df.to_csv('./Data/wuhan_int_airport_outbound_01012020_preprocess.csv')


### Top Outbound country from Wuhan Tianhe airport

In [72]:
df_selected = df[['From','To','Long','Lat','Op','FlightsCount','PosDateTime','Id', 'Icao','ICAO_code_from','ICAO_code_to']]

In [73]:
## rename the from-to --> source-target for gephi visualization & geo-map
df_selected = df_selected.rename(columns ={'From': 'Source', 'To': 'Target'})



In [74]:
df_selected.head()

,Source,Target,Long,Lat,Op,FlightsCount,PosDateTime,Id,Icao,ICAO_code_from,ICAO_code_to
0,"ZHHH Wuhan Tianhe, China","ZSPD Shanghai Pudong, China",121.24890,31.31760,Shanghai Airlines,6.0,2020-01-01 13:21:51.310,7869300.0,781374,ZHHH,ZSPD
1,"ZHHH Wuhan Tianhe, China","ZGGG Guangzhou Baiyun, China",113.44995,27.54648,China Southern Airlines,10.0,2020-01-01 13:22:09.775,7867910.0,780E06,ZHHH,ZGGG
2,"ZHHH Wuhan Tianhe, China","ZGGG Guangzhou Baiyun, China",113.44995,27.54648,China Southern Airlines,0.0,2020-01-01 13:22:06.952,7867910.0,780E06,ZHHH,ZGGG
3,"ZHHH Wuhan Tianhe, China","ZJHK Haikou Meilan, China",113.75509,22.70879,China Eastern Airlines,0.0,2020-01-01 13:21:32.470,7867564.0,780CAC,ZHHH,ZJHK
4,"ZHHH Wuhan Tianhe, China","ZSPD Shanghai Pudong, China",121.24890,31.31760,Shanghai Airlines,0.0,2020-01-01 13:21:49.668,7869300.0,781374,ZHHH,ZSPD


In [86]:
df.Id.value_counts()

7866120.0    953
7867910.0    241
7867564.0    226
7867548.0    225
7865422.0    198
7668741.0    194
7864725.0    167
7865074.0    165
7866856.0    163
7867644.0    157
7865599.0    154
8653999.0    130
7866467.0    125
8915023.0    117
8697816.0    114
7865204.0    111
7454084.0    108
7869300.0    108
7867627.0    107
7866533.0    104
7867820.0    104
7866122.0    102
7869832.0    102
7868854.0     99
7866216.0     98
7865836.0     97
7868342.0     97
7866241.0     97
7870355.0     96
7866454.0     94
7866629.0     94
7865336.0     93
7869665.0     85
7869831.0     84
7865076.0     82
7865353.0     78
7868483.0     76
7869022.0     72
7781702.0     71
7866647.0     69
8946005.0     69
8106103.0     69
7866753.0     68
7668869.0     64
7865325.0     63
7870388.0     62
7869599.0     56
8106177.0     53
7866510.0     53
7866853.0     51
7864440.0     47
7868358.0     45
8913995.0     45
7867342.0     42
7868927.0     39
7865059.0     37
9015365.0     33
7865598.0     28
7865283.0     